In [1]:

import sys
sys.path.append('/home/kasia/Python Projects/Medical-Appoitnment-No-Show')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random

from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import matthews_corrcoef

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef

from data_preprocessing.utils import preprocess_categorical_to_binary, change_age_range
from models.utils import split_train_test_with_unique_patient

pd.set_option('display.max_rows', 500)
%matplotlib inline


### Read the csv data to df

In [36]:
df = pd.read_csv('../data/raw/KaggleV2-May-2016.csv') 
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [ ]:
df.info()

### First information about dataset

In [ ]:
print(f'Dataset has {df.shape[0]} rows and {df.shape[1]} columns')

print(f'We see that one person has more then one appoitment, bacause we have {df.shape[0]} appointments and only {len(df.PatientId.unique())} persons')

len(df.AppointmentID.unique())
print('Appitment ID is unique. For every appoitnment we have new number. We can use this number in index')

In [ ]:
# percentage of woman with diabetes in group of 60
len(df[(df['Gender']=='F')&(df['Age']==60)&(df['Diabetes']==1)])/len(df[(df['Gender']=='F')&(df['Age']==60)])

In [ ]:
# percentage of woman with diabetes in group of 20
len(df[(df['Gender']=='F')&(df['Age']==20)&(df['Diabetes']==1)])/len(df[(df['Gender']=='F')&(df['Age']==20)])

In [ ]:
df[['Age','Diabetes','Alcoholism','Hipertension']].corr()


In [ ]:
df.head()

### categorical pipeline & numerical pipeline

In [ ]:
#train, test = split_train_test_with_unique_patient(df)


In [37]:
df = preprocess_categorical_to_binary(df)
df = change_age_range(df)

/home/kasia/Python Projects/Medical-Appoitnment-No-Show/src/data_preprocessing/utils.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][X[col]<0]=0
/home/kasia/Python Projects/Medical-Appoitnment-No-Show/src/data_preprocessing/utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][X[col]>95]=95


## correlaction gander vs target

 caterogies 0- woman
            1- men
            0- No
            1- Yes


In [56]:
# how many woman and how many man appear 
df.Gender[df['No-show']==0].value_counts()/df.Gender.value_counts()

0    0.796854
1    0.800321
Name: Gender, dtype: float64

In [57]:
df.Gender[df['No-show']==0].value_counts()

0    57246
1    30962
Name: Gender, dtype: int64

In [58]:
# how many woman and how many man didn't appear 
df.Gender[df['No-show']==1].value_counts()/df.Gender.value_counts()

0    0.203146
1    0.199679
Name: Gender, dtype: float64

In [ ]:
# matthews corrcoef is good for binary correlaction
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(train.Gender,train['No-show'])

correlation diffrence time sheduled and appoitment vs target

## correlation diffrence time sheduled and appoitment vs target & vs gender



calculate days between scheduled and appoitment. 
- check if we have collelation between days and target
- wenn the woman and the men do appoitment

In [ ]:
train_prep.groupby('DaysBetween')['Gender'].value_counts()
# we see that we have some data with -6 days what means that we should drop this data

In [ ]:
a = train_prep.groupby('DaysBetween')['Gender'].value_counts()

a.unstack(level=0)#.plot(kind='bar', subplots=False)

In [ ]:
cut_bins = [-0.001, 7, 14, 28, 84,168]
train_prep['days in week range']=pd.cut(train_prep['DaysBetween'],bins=cut_bins, labels = [1,2,4,12,24])

In [ ]:
#without normalization
sns.countplot(x='days in week range',hue='Gender',data=train_prep)


In [ ]:
aft_cut = train_prep.groupby(train_prep['days in week range'])['Gender'].value_counts()
aft_cut
aft_cut = aft_cut.groupby(level=1).apply(lambda x:x / float(x.sum()))
aft_cut 

#aft_cut.unstack().plot(kind='bar')


In [ ]:
#normalization for woman
xy = train_prep.groupby(train['ScheduledDay'].apply(lambda x :x.weekday))['Gender'].value_counts()
xy
state_pcts = xy.groupby(level=1).apply(lambda x:x / float(x.sum()))
state_pcts
state_pcts.unstack().plot(kind='bar')

In [ ]:
sns.countplot(x=train_prep['ScheduledDay'].apply(lambda x :x.month),hue='Gender',data=train_prep)

In [ ]:
zz = train_prep.groupby(train_prep['ScheduledDay'].apply(lambda x:x.month))['Gender'].value_counts()

zz.groupby(level=1).apply(lambda x: x/float(x.sum())).unstack().plot(kind='bar')

## Neighbourhood

In [ ]:
train.Neighbourhood.value_counts()

In [ ]:
zz = train.groupby(train['Neighbourhood'])['No-show'].value_counts()
zz
zz = zz.groupby(level=0).apply(lambda x: x/float(x.sum()))#.unstack().plot(kind='bar')##
zz
zz.loc[:,1].sort_values(ascending=False).head(40).plot(kind='bar')

## Age

In [ ]:
cut_bins = [-2, 20, 30, 40, 50,60,70,80,90,100,120]
train['age_after_cut']=pd.cut(train['Age'],bins=cut_bins, labels = [10, 20, 30, 40, 50,60,70,80,90,120])
train['age_after_cut'].value_counts().plot(kind='bar')


In [ ]:
#without normalization
ag = train.groupby(train['age_after_cut'])['Gender'].value_counts()
ag
#ag.unstack().plot(kind='bar')
#with normalization
ag.groupby(level=1).apply(lambda x: x/float(x.sum())).unstack().plot(kind='bar')

In [ ]:
ag.groupby(level=1).apply(lambda x: x/float(x.sum()))

## the older the people, the more sick they are?

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='Age', y ='Diabetes',hue='Gender',data=train,err_style='band')